# DeepEval Usage Demo (End-to-End)

Place your Crucible DeepEval export file in this same `notebooks/` folder, then set `DEEPEVAL_EXPORT_FILE` below.

In [ ]:
# Optional install
# %pip install -U deepeval requests


In [12]:
import json
from pathlib import Path

import requests

from deepeval import evaluate
from deepeval.test_case import LLMTestCase
from deepeval.metrics import AnswerRelevancyMetric, HallucinationMetric


In [13]:
NOTEBOOK_DIR = Path.cwd()
DEEPEVAL_EXPORT_FILE = 'crucible_rag_deepeval_20260220_172930.json'  # change this
DEEPEVAL_EXPORT_PATH = NOTEBOOK_DIR / DEEPEVAL_EXPORT_FILE

if not DEEPEVAL_EXPORT_PATH.exists():
    raise FileNotFoundError(f'Put export file in notebooks folder and update name: {DEEPEVAL_EXPORT_PATH}')

export_payload = json.loads(DEEPEVAL_EXPORT_PATH.read_text())
test_specs = export_payload['dataset']['test_cases']
len(test_specs)


25

In [14]:
APP_API_URL = 'http://localhost:8000/chat'  # change to your app endpoint

def call_app(question: str):
    resp = requests.post(APP_API_URL, json={'question': question}, timeout=30)
    resp.raise_for_status()
    data = resp.json()

    if isinstance(data, dict) and 'answer' in data:
        return data.get('answer', ''), data.get('contexts', []) or []

    return str(data), []


In [15]:
cases = []

for spec in test_specs:
    actual_output, retrieved_context = call_app(spec['input'])
    case = LLMTestCase(
        input=spec['input'],
        actual_output=actual_output,
        expected_output=spec.get('expected_output', ''),
        retrieval_context=retrieved_context if isinstance(retrieved_context, list) else [str(retrieved_context)],
    )
    cases.append(case)

metrics = [AnswerRelevancyMetric(threshold=0.7), HallucinationMetric(threshold=0.5)]
evaluation_result = evaluate(test_cases=cases, metrics=metrics)
evaluation_result


ConnectionError: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /chat (Caused by NewConnectionError("HTTPConnection(host='localhost', port=8000): Failed to establish a new connection: [Errno 61] Connection refused"))

In [ ]:
report_path = NOTEBOOK_DIR / 'deepeval_summary.txt'
report_path.write_text(str(evaluation_result))
report_path
